C# records only live in memory in a single process. If we want to take that idea of immutable records beyond the .NET runtime, we'll need some help. That's what Jinaga does.

We can store immutable records -- what Jinaga calls _facts_ -- in a local database. And we can share them with other computers. And we get the same semantics: two facts with the same type and fields are actually the same fact.

In [ ]:
#r "/Users/michaelperry/Projects/Jinaga.NET/Jinaga.UnitTest/bin/Debug/net5.0/Jinaga.UnitTest.dll"
using Jinaga;

[FactType("Jinaga.User")]
record User(string publicKey);

new User(publicKey)

publicKey
MIIBCgKCAQEAp36az4PJ6yNgvnCoaXt2bgZZ4Jce6Hl/zmPeKFE5jyfR7eXwuq3gZuF5DSRIdlcyYOrkawNxxVvWAR4f/tC72yCZsz9thBgu8TAeAAis5AKVGOmPvuXbbqpPG9pNIKlN1U+9EU+XepwS7++RF+amTzdN9+3CTKW1CBeSxeNgdLrLxliRsIuSQtbgcFzDmugFeHnXnj2LUdx08i3Xl/mDh6UHc1NwmtcCwPJYNe6m+dNmKWo1MpcnyYxO9bj922H/hTL30XXnIgQhEweDLLZHbqScZAVrT2yC+S+1+LIB14ZHfSpN0kEfBds9FKA9ZD+OLbTM75tysipMoVId/DjX1wIDAQAB


In [ ]:
using Jinaga.UnitTest;

var j = JinagaTest.Create();

var firstUser = await j.Fact(new User(publicKey));
var secondUser = await j.Fact(new User(publicKey));

firstUser.Equals(secondUser)

True

In [ ]:
[FactType("Corporate.Company")]
record Company(string identifier);

[FactType("Corporate.Employee")]
record Employee(Company company, int employeeNumber);

var contoso = await j.Fact(new Company("Contoso"));
var jane = await j.Fact(new Employee(contoso, 1));
var bob = await j.Fact(new Employee(contoso, 2));

var employees = await j.Query(contoso, Given<Company>.Match((company, facts) =>
    facts.OfType<Employee>().Where(employee => employee.company == company)
));
employees

index company employeeNumber 0 identifier Contoso 1 1 identifier Contoso 2